In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Contingency Timings Appendix (Feb 24, 2025) 

<h2 style="height: 8px; background-color: #003262; width: 95%;"></h2>
<h2 style="height: 4px; background-color: #FDB515; width: 90%; margin-bottom: 0px;"></h2>

<div style="text-align: center;">
  <img src = "Art/ASUC_Senator_Ngai_Artboard 3.png" style="width:700px;height:700px">
</div>

## Section 0: Cleaning Functions

In [2]:
_valid_iterables = (list, tuple, pd.Series, np.ndarray, pd.Index)

def get_valid_iter():
    return _valid_iterables

def _is_type(inpt, t):
    # private function
    """
    Private helper function to check if an input is of a specified type or, if iterable, 
    whether all elements belong to at least one specified type.

    Args:
        inpt: The input value or iterable of values to be checked.
        t: A single type or an iterable of types to validate against.

    Returns:
        bool: True if `inpt` matches at least one of the specified types, 
              or if `inpt` is an iterable and all its elements match at least one type in `t`. 
              False otherwise.
    """
    def _is_type_helper(inpt, t):
        """
        Checks if the input is of a specified type `t` or, if an iterable, 
        whether all elements in `inpt` are of type `t`.
        """
        if isinstance(inpt, get_valid_iter()) and len(inpt) == 0:
            raise ValueError("Input iterable to check types of is an empty iterable.")
        return isinstance(inpt, t) or (isinstance(inpt, get_valid_iter()) and all(isinstance(x, t) for x in inpt))
    
    if isinstance(t, get_valid_iter()):
        if len(t) == 0:
            raise ValueError("Type input 't' is an empty iterable.")
        return any(_is_type_helper(inpt, type) for type in t) #was previously all
    else:
        return _is_type_helper(inpt, t)
    
def is_type(inpt, t):
    """
    Public function to check if an input is of a specified type or, if iterable, 
    whether all elements belong to at least one specified type.

    Args:
        inpt: The input value or iterable of values to be checked.
        t: A single type or an iterable containing multiple types to validate against.

    Returns:
        bool: 
            - True if `inpt` is of type `t`.
            - True if `inpt` is an iterable and all its elements match at least one type in `t`.
            - False otherwise.

    Examples:
        >>> is_type(5, int)
        True
        
        >>> is_type([1, 2, 3], int)
        True
        
        >>> is_type(["hello", 3], (int, str))
        True
        
        >>> is_type(["hello", 3], int)
        False
    """
    
    return _is_type(inpt, t)
    
def _in_df(inpt, df):
    #private function
    """
    Private function to check if a given input (column label or index) exists in a DataFrame.

    Args:
        inpt: A string (column label), an integer (column index), or an iterable (tuple, list, or pd.Series) of strings or integers.
        df (pd.DataFrame): The DataFrame to check against.

    Returns:
        bool: 
            - True if `inpt` is a column label in `df` (if `inpt` is a string).
            - True if `inpt` is a valid column index in `df` (if `inpt` is an integer and non-negative).
            - True if all elements in `inpt` exist as column labels in `df` (if `inpt` is an iterable of strings).
            - True if all elements in `inpt` are valid column indices in `df` (if `inpt` is an iterable of non-negative integers).
            - False otherwise.

    Raises:
        AssertionError: If `inpt` is not a string, integer, or an iterable of strings/integers.
        AssertionError: If `inpt` is a negative integer.
    """
    assert is_type(inpt, (str, int)), 'inpt must be string, int or tuple, list or pd.Series of strings or ints.'
    if isinstance(inpt, str): 
        return inpt in df.columns
    elif isinstance(inpt, int):
        assert inpt >= 0, 'integer inpt values must be non-negative.'
        return inpt < len(df.columns)
    elif isinstance(inpt[0], str):
        return pd.Series(inpt).isin(df.columns).all()
    elif isinstance(inpt[0], int):
        return all(pd.Series(inpt) < len(df.columns))


def in_df(inpt, df):
    """
    Public function to check if a given input (column label or index) exists in a DataFrame.

    This function wraps `_in_df()`.

    Args:
        inpt: A string (column label), an integer (column index), or an iterable (tuple, list, or pd.Series) of strings or integers.
        df (pd.DataFrame): The DataFrame to check against.

    Returns:
        bool: True if `inpt` exists in the DataFrame as a column label or index, False otherwise.

    Examples:
        >>> in_df("ColumnA", df)
        True
        
        >>> in_df([0, 2, 4], df)
        True
    """
    return _in_df(inpt, df)
    
def _any_in_df(inpt, df):
    #private function
    """
    Private function to check if at least one column in an iterable exists in a DataFrame.

    This function does not handle integers because DataFrame shape can be used to check if an index exists.

    Args:
        inpt: A string (column label) or an iterable (tuple, list, or pd.Series) of strings.
        df (pd.DataFrame): The DataFrame to check against.

    Returns:
        bool: 
            - True if `inpt` is a column label in `df` (if `inpt` is a string).
            - True if at least one element in `inpt` exists as a column label in `df` (if `inpt` is an iterable of strings).
            - False otherwise.

    Raises:
        AssertionError: If `inpt` is not a string or an iterable of strings.
    """
    assert is_type(inpt, str), 'inpt must be string or tuple, list or pd.Series of strings.'
    if isinstance(inpt, str): 
        return inpt in df.columns
    else:
        return any(df.columns.isin(inpt))
    
def any_in_df(inpt, df):
    """
    Public function to check if at least one column in an iterable exists in a DataFrame.

    This function wraps `_any_in_df()`.

    Args:
        inpt: A string (column label) or an iterable (tuple, list, or pd.Series) of strings.
        df (pd.DataFrame): The DataFrame to check against.

    Returns:
        bool: True if at least one element in `inpt` exists in the DataFrame as a column label, False otherwise.

    Examples:
        >>> any_in_df("ColumnA", df)
        True
        
        >>> any_in_df(["ColumnA", "NonexistentColumn"], df)
        True
    """
    return _any_in_df(inpt, df)
    
def _concatonater(input_df, base_df, sort_cols=None):
    #private
    output = pd.concat([input_df, base_df])
    if sort_cols is not None:
        assert is_type(sort_cols, str), 'sort_cols must be string or a list of strings'
        assert in_df(list(sort_cols), base_df) or in_df(list(sort_cols), input_df), 'Column/some columns in sort_cols not in input_df or base_df.'
        
        output = output.sort_values(by=sort_cols, ascending=False)
    
    return output

def concatonater(input_df, base_df, sort_cols=None):
    return _concatonater(input_df, base_df, sort_cols)

def _academic_year_parser(inpt):
    #private
    def _academic_year_helper(timestamp):
        """Takes timestamp and returns academic year"""
        if timestamp.month > 7:
            return str(timestamp.year) + "-" + str(timestamp.year + 1)
        elif timestamp.month < 6:
            return str(timestamp.year - 1) + "-" + str(timestamp.year)
        else:
            raise ValueError("The input timestamp occurs in June or July, which are typically not part of a standard academic year.")
        
    def _validate_and_parse(data):
        """Validates and parses collections of timestamps."""
        try:
            timestamps = pd.Series(data).apply(pd.Timestamp)
        except Exception:
            raise ValueError("At least one input could not be converted to a valid timestamp.")
        if not timestamps.apply(lambda x: hasattr(x, "month") and hasattr(x, "year")).all():
            raise ValueError("At least one timestamp does not include both month and year attributes.")
        return timestamps.apply(_academic_year_helper)
        
    if isinstance(inpt, pd.Timestamp):
        return _academic_year_helper(inpt)
    elif isinstance(inpt, str):
        try:
            inpt = pd.Timestamp(inpt)
        except Exception as e:
            raise ValueError('inpt could not be converted to valid timestamp')
        if hasattr(inpt, "month") and hasattr(inpt, "year"):
            return _academic_year_helper(inpt)
        else:
            raise ValueError("The timestamp must include both month and year attributes.")
    elif isinstance(inpt, get_valid_iter()):
        return _validate_and_parse(inpt)
    else:
        raise ValueError("Input must be a string, pd.Timestamp, or a list, tuple, or pd.Series containing strings or pd.Timestamps.")
    
def academic_year_parser(inpt):
    """Takes in a date and returns which academic year it's a part of."""
    return _academic_year_parser(inpt)

def _reverse_academic_year_parser(inpt, year_start_end):
    assert is_type(inpt, str), 'Input must be a string or list of strings specifying academic year'

    def _acayear_instance_processor(inpt, year_start_end):
        if '-' in inpt:
            years = inpt.split('-')
            start = pd.Timestamp(f"{years[0]}-{year_start_end[0][0]}-{year_start_end[0][1]}")
            end = pd.Timestamp(f"{years[1]}-{year_start_end[1][0]}-{year_start_end[1][1]}")
        elif 'fy' in inpt.lower():
            end_year = f"20{inpt[2:]}"
            start_year = str(int(end_year) - 1)
            start = pd.Timestamp(f"{start_year}-{year_start_end[0][0]}-{year_start_end[0][1]}")
            end = pd.Timestamp(f"{end_year}-{year_start_end[1][0]}-{year_start_end[1][1]}")
        else:
            raise ValueError('Inputted academic year belongs to unkown format.')
        return (start, end)
    if isinstance(inpt, str):
        return _acayear_instance_processor(inpt, year_start_end)
    elif isinstance(inpt, get_valid_iter()):
        return list(_reverse_academic_year_parser(np.array(inpt)))
    else: 
        raise ValueError("Input must be a string, or a list os strings indicating academic year in a known format.")

def reverse_academic_year_parser(inpt, year_start_end=((8, 15), (5, 20))):
    """Takes in a academic year and returns a tuple indicating date range that pertains to said academic year.
    If there are multiple academic years, returns as a list of 2d tuples.

    year_start_end: requires you to specify the month and date of the start of the academic year and end of 
    the academic year in a nested tuple format ((start_month, start_day), (end_month, end_day))
    
    Handles following formats:
    - year-year (eg. 2023-2024)
    - FY## (eg. FY24 corrsponding to 2023-2024)"""
    return _reverse_academic_year_parser(inpt, year_start_end)

In [3]:
def sucont_cleaner(df, year):
    """Version 1.0: Just handles cleaning for years"""
    assert ('Date' in df.columns) and is_type(df['Date'], pd.Timestamp), 'df must have "Date" column that contains only pd.Timestamp objects'
    
    copy = df.copy()
    year_range = reverse_academic_year_parser(year)
    mask = (copy['Date'] >= year_range[0]) & (copy['Date'] <= year_range[1])
    return pd.DataFrame(copy[mask]).reset_index()

<h2 style="height: 6px; background-color: #003262; width: 95%;"></h2>
<h2 style="height: 2px; background-color: #FDB515; width: 90%; margin-bottom: 0px;"></h2>

## Section 1: Loading Data

In [4]:
SUContingency = pd.read_csv('Cleaned Datasets/SUContingency Master.csv', parse_dates=['Date'])

In [5]:
cont_df1 = SUContingency[SUContingency['Date'].dt.year == 2024]
cont_df2 = SUContingency[(SUContingency['Date'].dt.year == 2023) & (SUContingency['Date'].dt.month > 7)]
Contingency_2324 = pd.concat([cont_df2, cont_df1, ])

raw_cop = Contingency_2324.copy()

Contingency_2324['Recipient'] = Contingency_2324['Memo'].str.extract(r'[FR|SR]\s\d{2}\/\d{2}[-\s](?:[F|S]\d{2}\s-\s|\d+\s)(.+)') #isolating recipient from `memo` column

#changing the way senate offices and EVP are referred to for consistency 
Contingency_2324['Recipient'] = Contingency_2324['Recipient'].apply(lambda x: 'ASUC ' + x if (type(x) is str) and ('Office of Senator' in x) else x) #make sure type check goes first or else function tries to check membership against NaN values which are floats
Contingency_2324['Recipient'] = Contingency_2324['Recipient'].apply(lambda x: 'ASUC - Office of the Executive Vice President' if (type(x) is str) and (x == 'ASUC EVP') else x) 
Contingency_2324_initial = Contingency_2324.reset_index()

In [6]:
#checks
print(f"Amounts column matches: {all(sucont_cleaner(SUContingency, 'FY24')['Amount'] == Contingency_2324_initial['Amount'])}")
print(f"Balance column matches: {all(sucont_cleaner(SUContingency, 'FY24')['Available Balance After'] == Contingency_2324_initial['Available Balance After'])}")
print(f"Date column matches: {all(sucont_cleaner(SUContingency, 'FY24')['Date'] == Contingency_2324_initial['Date'])}")
print(f"Memo column matches: {all(sucont_cleaner(SUContingency, 'FY24')['Memo'] == Contingency_2324_initial['Memo'])}")


Amounts column matches: True
Balance column matches: True
Date column matches: True
Memo column matches: True


In [7]:
Non_Admin_Contin = SUContingency[SUContingency['Admin'] == 0]

In [8]:
Non_Admin_Contin2324 = sucont_cleaner(Non_Admin_Contin, 'FY24')

In [9]:
Contingency_2324 = sucont_cleaner(SUContingency, 'FY24')

In [10]:
Contingency_2324.head(5)

,index,Account Name,Account Description,Transaction Reference #,Date,Reconciled,Created By,Payee/Source First Name,Payee/Source Last Name,Originator Account Name,...,From Request - Payee ZIP,Memo,Category,Type,Transaction Method,Amount,Ending Balance After,Available Balance After,Admin,Recipient
0,441,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1837928,2023-09-21,Yes,Henry F Isselbacher,Transfer from '3-40-100220-00000-ASUCFunds-SCF...,NaN,3-40-100220-00000-ASUCFunds-SCF,...,NaN,FR 23/24 F05 - Azaad,Transfer between Accounts,NaN,Payment,1500.0,0.0,98500.0,0,Azaad
1,442,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1837929,2023-09-21,Yes,Henry F Isselbacher,Transfer from '3-40-100220-00000-ASUCFunds-SCF...,NaN,3-40-100220-00000-ASUCFunds-SCF,...,NaN,FR 23/24 F05 - Pi Sigma Epsilon,Transfer between Accounts,NaN,Payment,600.0,0.0,97900.0,0,Pi Sigma Epsilon
2,443,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1838276,2023-09-28,Yes,Henry F Isselbacher,Transfer from '3-40-100220-00000-ASUCFunds-SCF...,NaN,3-40-100220-00000-ASUCFunds-SCF,...,NaN,FR 23/24 F05 - Office of Senator Helena Wu,Transfer between Accounts,NaN,Payment,284.8,0.0,97615.2,0,ASUC Office of Senator Helena Wu
3,444,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1838277,2023-09-28,Yes,Henry F Isselbacher,Transfer from '3-40-100220-00000-ASUCFunds-SCF...,NaN,3-40-100220-00000-ASUCFunds-SCF,...,NaN,FR 23/24 F05 - Office of Senator Andy Liu,Transfer between Accounts,NaN,Payment,390.0,0.0,97225.2,0,ASUC Office of Senator Andy Liu
4,445,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1838400,2023-09-29,Yes,Robert Jittrikawiphol,Transfer from '3-40-100220-00000-ASUCFunds-SCF...,NaN,3-40-100220-00000-ASUCFunds-SCF,...,NaN,FR 23/24 Fall Week 6 Senate Contingency Alloca...,Transfer between Accounts,NaN,Payment,284.8,0.0,96940.4,0,NaN


In [11]:
SUContingency.head(5)

,Account Name,Account Description,Transaction Reference #,Date,Reconciled,Created By,Payee/Source First Name,Payee/Source Last Name,Originator Account Name,From Request - Account Name,...,From Request - Payee ZIP,Memo,Category,Type,Transaction Method,Amount,Ending Balance After,Available Balance After,Admin,Recipient
0,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1705233,2016-08-17,No,NaN,Reversal of Year End 15-16 Balance,NaN,3-40-100220-00000-ASUCFunds-SCF,NaN,...,NaN,Reversal of Year End 15-16 Balance,X-Adjustment (Admin use only),Direct Deposit ACH/Stripe,Payment,25080.28,0.0,0.0,1,NaN
1,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1705287,2016-08-17,No,NaN,FY 16-17 Budget Allocation - Senate Contingenc...,NaN,3-40-100220-00000-ASUCFunds-SCF,NaN,...,NaN,FY 16-17 Budget Allocation - Senate Contingenc...,X-Adjustment (Admin use only),Direct Deposit ACH/Stripe,Deposit,80000.00,0.0,80000.0,1,NaN
2,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1705571,2016-08-24,No,Robert Jittrikawiphol,Transfer from '3-40-203100-00000-ASUCFunds-UAV...,NaN,3-40-203100-00000-ASUCFunds-UAVA-B,NaN,...,NaN,"ESC grouped organization , returning funds bac...",X-Adjustment (Admin use only),NaN,Deposit,400.00,0.0,80400.0,1,NaN
3,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1707772,2016-10-06,No,VIRGIL TANG,Sheena,Paul,3-40-100220-00000-ASUCFunds-SCF,3-40-100220-00000-ASUCFunds-SCF,...,94609.0,VT 9/28/2016 CK#122570,Stipend,Check,Payment,200.00,0.0,80200.0,0,NaN
4,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1707841,2016-10-06,No,NaN,Robert Jittrikawiphol,NaN,3-40-100220-00000-ASUCFunds-SCF,NaN,...,NaN,"October 5, 2016 ASUC Senate Contingency Fund A...",Transfer between Accounts,Internal Allocation/Transaction/Transfer,Payment,4020.00,0.0,76180.0,0,NaN


<h2 style="height: 6px; background-color: #003262; width: 95%;"></h2>
<h2 style="height: 2px; background-color: #FDB515; width: 90%; margin-bottom: 0px;"></h2>

## Section 2: Initial Statistics

In [12]:
Contingency_2324['Amount'].describe()

count       99.000000
mean       832.734040
std       2018.609419
min       -470.000000
25%        250.000000
50%        432.000000
75%        660.000000
max      18000.000000
Name: Amount, dtype: float64

In [13]:
sum(Contingency_2324['Amount'])

82440.67000000001

In [14]:
SUContingency['Amount'].describe()

count      540.000000
mean      1696.945630
std       7318.897354
min      -1500.000000
25%        299.750000
50%        500.000000
75%       1200.000000
max      93542.000000
Name: Amount, dtype: float64

<h2 style="height: 6px; background-color: #003262; width: 95%;"></h2>
<h2 style="height: 2px; background-color: #FDB515; width: 90%; margin-bottom: 0px;"></h2>

## Section 3: Figures

In [15]:
fig_1 = px.line(Non_Admin_Contin2324, x="Date", y="Available Balance After", 
                title='SU Contingency Funding Total Balance over Time (FY24)')
fig_1.show()

In [16]:
full_allocation_balance = Non_Admin_Contin.groupby('Date')['Available Balance After'].agg('min').reset_index()
fig_2 = px.line(full_allocation_balance, x="Date", y="Available Balance After", title='SU Contingency Total $Balance over Time (Since 2016)')
fig_2.show()

In [17]:
allocation_amounts = Non_Admin_Contin2324[Non_Admin_Contin2324['Admin'] == 0].groupby('Date')['Amount'].agg('sum').reset_index()
fig_3 = px.line(allocation_amounts, x="Date", y="Amount", title='SU Contingency $Amt Allocations over Time (FY24)')
fig_3.show()

In [18]:
full_allocation_amounts = Non_Admin_Contin.groupby('Date')['Amount'].agg('sum').reset_index()
fig_4 = px.line(full_allocation_amounts, x="Date", y="Amount", title='SU Contingency Total $Amt Allocations over Time (Since 2016)')
fig_4.show()

In [19]:
full_allocation_amounts_no_outliers = full_allocation_amounts[full_allocation_amounts['Date'] != pd.Timestamp('2019-09-26')]
fig_5 = px.line(full_allocation_amounts_no_outliers, x="Date", y="Amount", title='SU Contingency Total $Amt Allocations over Time (Since 2016) NO OUTLIERS')
fig_5.show()

In [20]:
excluded = Non_Admin_Contin[Non_Admin_Contin['Date'] != pd.Timestamp('2019-09-26')]

past_5k = excluded[(excluded['Amount'] > 5000) & (excluded['Amount'] <= 10000)].copy()
past_5k['Category'] = '5K+'

past_10k = excluded[excluded['Amount'] > 10000].copy()
past_10k['Category'] = '10K+'

past_2_5k = excluded[(excluded['Amount'] > 2500) & (excluded['Amount'] <= 5000)].copy()
past_2_5k['Category'] = '2.5K+'

past_1_5k = excluded[(excluded['Amount'] > 1500) & (excluded['Amount'] <= 2500)].copy()
past_1_5k['Category'] = '1.5K+'

all_data = pd.concat([past_5k, past_10k, past_2_5k, past_1_5k])

fig_6 = px.scatter(
    all_data,
    x="Date",
    y="Amount",
    color="Category",  
    labels={'x': 'Date', 'y': 'Amount'},
    title="Spikes in Amounts Across Time (Since 2016)"
)

fig_6.show()

<h2 style="height: 6px; background-color: #003262; width: 95%;"></h2>
<h2 style="height: 2px; background-color: #FDB515; width: 90%; margin-bottom: 0px;"></h2>

## Section 4: Spike Data

In [21]:
excluded = Non_Admin_Contin[Non_Admin_Contin['Date'] != pd.Timestamp('2019-09-26')]

past_5k = excluded[excluded['Amount'] > 5000]
past_10k = excluded[excluded['Amount'] > 10000]
past_2_5k = excluded[excluded['Amount'] > 2500]
past_1_5k = excluded[excluded['Amount'] > 1500]

print(f"Amount of spikes past 5k: {len(past_5k)}")
print(f"Amount of spikes past 10k: {len(past_10k)}")
print(f"Amount of spikes past 2.5k: {len(past_2_5k)}")
print(f"Amount of spikes past 1.5k: {len(past_1_5k)}")

Amount of spikes past 5k: 6
Amount of spikes past 10k: 3
Amount of spikes past 2.5k: 22
Amount of spikes past 1.5k: 42


In [22]:
print(f"Amount of spikes past 5k in FY24: {len(Contingency_2324[(Contingency_2324['Amount'] > 5000) & (Contingency_2324['Amount'] <= 10000)])}")
print(f"Amount of spikes past 10k in FY24: {len(Contingency_2324[Contingency_2324['Amount'] > 10000])}")
print(f"Amount of spikes past 2.5k in FY24: {len(Contingency_2324[(Contingency_2324['Amount'] > 2500) & (Contingency_2324['Amount'] <= 5000)])}")
print(f"Amount of spikes past 1.5k in FY24: {len(Contingency_2324[(Contingency_2324['Amount'] > 1500) & (Contingency_2324['Amount'] <= 2500)])}")

Amount of spikes past 5k in FY24: 1
Amount of spikes past 10k in FY24: 1
Amount of spikes past 2.5k in FY24: 0
Amount of spikes past 1.5k in FY24: 1


In [23]:
print(f"Total amount accounted for by Spikes in FY24: {sum(Contingency_2324[Contingency_2324['Amount'] > 1500]['Amount'])}")
print(f"Total percentage amount accounted for by Spikes in FY24: {sum(Contingency_2324[Contingency_2324['Amount'] > 1500]['Amount'])/100000 * 100}")

Total amount accounted for by Spikes in FY24: 29825.14
Total percentage amount accounted for by Spikes in FY24: 29.82514


### Additional Info: What happened in September of 2019?

In [24]:
Non_Admin_Contin[Non_Admin_Contin['Date'] == pd.Timestamp('2019-09-26')]

,Account Name,Account Description,Transaction Reference #,Date,Reconciled,Created By,Payee/Source First Name,Payee/Source Last Name,Originator Account Name,From Request - Account Name,...,From Request - Payee ZIP,Memo,Category,Type,Transaction Method,Amount,Ending Balance After,Available Balance After,Admin,Recipient
118,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1762223,2019-09-26,No,Robert Jittrikawiphol,Transfer from '3-40-100256-00000-ASUC Funds-AS...,NaN,3-40-100256-00000-ASUC Funds-ASUC CI,NaN,...,NaN,tfr to Senate Contingency per FR 19/20-08 Fall...,Transfer between Accounts,NaN,Deposit,30000.0,0.0,29608.0,0,Fall 2019
119,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1762224,2019-09-26,No,Robert Jittrikawiphol,Transfer from '3-41-100256-00000-Contingency C...,NaN,r* 3-41-100256-00000-Contingency CASSA Funds,NaN,...,NaN,tfr per FR 19/20-08 Fall 2019,Transfer between Accounts,NaN,Deposit,60000.0,0.0,89608.0,0,Fall 2019
120,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1762225,2019-09-26,No,Robert Jittrikawiphol,Transfer from '3-40-100220-00000-ASUCFunds-SCF...,NaN,3-40-100220-00000-ASUCFunds-SCF,NaN,...,NaN,FR 19/20 Fall Week 3-Danceworx,Transfer between Accounts,NaN,Payment,750.0,0.0,88858.0,0,NaN
121,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1762226,2019-09-26,No,Robert Jittrikawiphol,Transfer from '3-40-100220-00000-ASUCFunds-SCF...,NaN,3-40-100220-00000-ASUCFunds-SCF,NaN,...,NaN,FR 19/20 Fall Week 3-Danceworx,Transfer between Accounts,NaN,Payment,450.0,0.0,88408.0,0,NaN
122,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1762227,2019-09-26,No,Robert Jittrikawiphol,Transfer from '3-40-100220-00000-ASUCFunds-SCF...,NaN,3-40-100220-00000-ASUCFunds-SCF,NaN,...,NaN,FR 19/20 Fall Week 3-ASRB,Transfer between Accounts,NaN,Payment,485.0,0.0,87923.0,0,NaN
123,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1762226,2019-09-26,No,Robert Jittrikawiphol,Transfer from '3-40-100220-00000-ASUCFunds-SCF...,NaN,3-40-100220-00000-ASUCFunds-SCF,NaN,...,NaN,CANCELLED Transaction #:1762226,NaN,NaN,Payment,-450.0,0.0,88373.0,0,NaN
124,3-40-100220-00000-ASUCFunds-SCF,ASUC/GA Student Fee Funding,1762228,2019-09-26,No,Robert Jittrikawiphol,Transfer from '3-40-100220-00000-ASUCFunds-SCF...,NaN,3-40-100220-00000-ASUCFunds-SCF,NaN,...,NaN,FR 19/20 Fall Week 3-DWBB,Transfer between Accounts,NaN,Payment,450.0,0.0,87923.0,0,NaN
